## Real Time Location System

### Predicting Location with K- Nearest Neighbor

### Team: Bin Yu, Nuoya Rezsonya  (add your name here)

### Introduction ( we will add this part later)

Introduction part

### Methodology (we will add this part later)

Talking about what we used in this notebook and reference the resource 

### Data Import

This part will be words to describe the data source as in where it comes from, dimension of the data, column explaination(which is what)

In [4]:
pwd

ERROR: Error in eval(expr, envir, enclos): object 'pwd' not found


In [5]:
# Import the data, please dont remove my import, you can comment out and add yours
txt <- readLines( "data\\offline_final_trace.txt")


Warning message in file(con, "r"):
"cannot open file 'data\offline_final_trace.txt': No such file or directory"

ERROR: Error in file(con, "r"): cannot open the connection


In [27]:
print(paste("The total number of lines in offline data =",length(txt)))
print(paste("The number of comments in data =", sum(substr(txt, 1, 1) == "#"))) # there are comments in the data, figure out amount
print(paste("The actual number of data, non-comment lines =", length(txt) - sum(substr(txt, 1, 1) == "#"))) # the actual data lines


[1] "The total number of lines in offline data = 151392"
[1] "The number of comments in data = 5312"
[1] "The actual number of data, non-comment lines = 146080"
